In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from tqdm import tqdm

In [ ]:
input_size = 28
sequence_length = 28
num_layers = 2
hidden_size = 256
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 2

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, num_classes):
    super(RNN, self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
    self.fc = nn.Linear(hidden_size*sequence_length, num_classes)

  def forward(self, x):
    h0 = torch.zeros(
        self.num_layers,
        x.size(0),
        self.hidden_size
    ).to(device)
    # forward pass
    out, _ = self.rnn(x, h0)
    out = out.reshape(out.shape[0], -1)
    out = self.fc(out)
    return out

class GRU(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, num_classes):
    super(GRU, self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
    self.fc = nn.Linear(hidden_size*sequence_length, num_classes)

  def forward(self, x):
    h0 = torch.zeros(
        self.num_layers,
        x.size(0),
        self.hidden_size
    ).to(device)
    # forward pass
    out, _ = self.gru(x, h0)
    out = out.reshape(out.shape[0], -1)
    out = self.fc(out)
    return out

class LSTM(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, num_classes):
    super(LSTM, self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
    self.fc = nn.Linear(hidden_size*sequence_length, num_classes)

  def forward(self, x):
    h0 = torch.zeros(
        self.num_layers,
        x.size(0),
        self.hidden_size
    ).to(device)
    c0 = torch.zeros(
        self.num_layers,
        x.size(0),
        self.hidden_size
    ).to(device)
    # forward pass
    out, _ = self.lstm(x, (h0, c0))
    out = out.reshape(out.shape[0], -1)
    out = self.fc(out)
    return out

In [ ]:
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)

In [ ]:
train_dataset

Dataset MNIST
    Number of datapoints: 60000
    Root location: dataset/
    Split: Train
    StandardTransform
Transform: ToTensor()

In [ ]:
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
train_loader

In [ ]:
test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_dataset

Dataset MNIST
    Number of datapoints: 10000
    Root location: dataset/
    Split: Test
    StandardTransform
Transform: ToTensor()

In [ ]:
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)
test_loader

In [ ]:
model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

In [ ]:
model = GRU(input_size, hidden_size, num_layers, num_classes).to(device)

In [ ]:
model = LSTM(input_size, hidden_size, num_layers, num_classes).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)

In [ ]:
for epoch in tqdm(range(num_epochs)):
  for batch_idx, (data, targets) in enumerate(train_loader):
    # Get the data to cuda if possible
    data = data.to(device=device).squeeze(1) # 64, 1, 28, 28
    targets = targets.to(device=device)

    # Forward pass
    scores = model(data)

    # Calculate the loss
    loss = criterion(scores, targets)

    # Backward
    optimizer.zero_grad()
    loss.backward()

    # gradient descent or adam step
    optimizer.step()


100%|██████████| 2/2 [00:19<00:00,  9.70s/it]


In [ ]:
def check_accuracy(loader, model):
  if loader.dataset.train:
    print("Checking accuracy on training data")
  else:
    print("Checking accuracy on test data")

  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad():
    for x, y in loader:
      x = x.to(device=device).squeeze(1)
      y = y.to(device=device)
      # x = x.reshape(x.shape[0], -1)

      scores = model(x)
      _, predictions = scores.max(dim=1)

      num_correct += (predictions == y).sum()
      num_samples += predictions.size(0)

    print(f"Got {num_correct}/{num_samples} with accuracy {float(num_correct)/float(num_samples)*100}")

  model.train()

In [ ]:
check_accuracy(train_loader, model) # RNN

Checking accuracy on training data
Got 58215/60000 with accuracy 97.02499999999999


In [ ]:
check_accuracy(test_loader, model) # RNN

Checking accuracy on test data
Got 9693/10000 with accuracy 96.93


In [ ]:
check_accuracy(train_loader, model) ## GRU

Checking accuracy on training data
Got 59245/60000 with accuracy 98.74166666666667


In [ ]:
check_accuracy(test_loader, model) # GRU

Checking accuracy on test data
Got 9848/10000 with accuracy 98.48


In [ ]:
check_accuracy(train_loader, model) ## LSTM

Checking accuracy on training data
Got 59178/60000 with accuracy 98.63


In [ ]:
check_accuracy(test_loader, model) ## LSTM

Checking accuracy on test data
Got 9835/10000 with accuracy 98.35000000000001
